# Iterative Construction of a Penalised Vine Structure
This notebook iteratively estimate the quantile.

#### Libraries

In [1]:
import openturns as ot
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

random_state = 123
np.random.seed(random_state)

#### Model function
This example consider the simple additive example.

In [2]:
from dependence.tests import func_overflow, margins_overflow, var_names_overflow, func_sum

test_func = func_overflow

#### Dimension and margins
We first define the problem dimension and the margins

In [3]:
if test_func == func_overflow:
    margins = margins_overflow
    dim = len(margins)
else:
    dim = 6
    margins = [ot.Normal()]*dim

We chose the coefficients of the variables throught the additive function.

In [4]:
if test_func == func_sum:
    coeficients = np.logspace(0., 5., dim+1, endpoint=False)[1:]

    n_plot = 10000
    x = np.asarray(ot.ComposedDistribution(margins).getSample(n_plot))
    y = test_func(x, coeficients)

    fig, axes = plt.subplots(dim, 1, sharex=True, sharey=True, figsize=(4, 2*dim))
    for i in range(dim):
        ax = axes[i]
        ax.plot(x[:, i], y, '.')
        ax.set_xlabel(r'$X_%d$' % (i+1), fontsize=12)
        ax.set_ylabel(r'$y$', fontsize=12)
    fig.tight_layout()

#### Copula families
We now consider only Gaussian dependencies for this example

In [5]:
families = np.zeros((dim, dim), dtype=int)
for i in range(1, dim):
    for j in range(i):
        families[i, j] = 1

## Estimations
We create an instance of the main class for conservative estimate, and we define a q_func object for the quantile as a quantity of interest

In [6]:
from dependence import ConservativeEstimate, quantile_func

alpha = 0.95

if alpha > 0.5: # Maximizing the quantile
    def q_func(x, axis=1):
        return - quantile_func(alpha)(x, axis=axis)
else: # Minimizing
    q_func = quantile_func(alpha)

quant_estimate = ConservativeEstimate(model_func=test_func, margins=margins, families=families)

First, we compute the quantile at independence

In [7]:
n = 10000
indep_result = quant_estimate.independence(n_input_sample=n, q_func=q_func, random_state=random_state)[0]

In [8]:
indep_result.compute_bootstrap(1000)
boot_std = indep_result.bootstrap_sample.std()
boot_mean = indep_result.bootstrap_sample.mean()
print('Quantile at independence: %.2f with a C.O.V at %.1f %%' % (boot_mean, abs(boot_std/boot_mean)*100.))

Quantile at independence: 4.86 with a C.O.V at 2.1 %


### Iterative Approach
Now lets see how good we can be with the iterative appraoch.

In [14]:
from dependence import iterative_vine_minimize

n = 10000
p_max = 3
K = None
verbose = True
grid_type = 'vertices'
n_add_pairs = 1
n_remove_pairs = 1
iterative_save = "./results/iterative_save"
iterative_load = True

quant_estimate = ConservativeEstimate(model_func=test_func, margins=margins, families=families)

results = iterative_vine_minimize(quant_estimate, 
                                                                          n_input_sample=n,
                                                                          n_dep_param_init=K,
                                                                          max_n_pairs=p_max, 
                                                                          grid_type=grid_type, 
                                                                          q_func=q_func,
                                                                          n_add_pairs=n_add_pairs,
                                                                          n_remove_pairs=n_remove_pairs,
                                                                          verbose=True, 
                                  iterative_save=iterative_save,
                                 iterative_load=iterative_load)

Data saved in ./results/iterative_save1_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0.hdf
Worst quantile of [(1L, 0L)] at 1.61890855153
Data saved in ./results/iterative_save0_1_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0.hdf
Worst quantile of [(2L, 0L)] at 3.8023413144
Data saved in ./results/iterative_save0_0_1_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0.hdf
Worst quantile of [(2L, 1L)] at 4.39044839137
Data saved in ./results/iterative_save0_0_0_1_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0.hdf
Worst quantile of [(3L, 0L)] at 4.43397312782
Data saved in ./results/iterative_save0_0_0_0_1_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0.hdf
Worst quantile of [(3L, 1L)] at 4.82463359712
Data saved in ./results/iterative_save0_0_0_0_0_1_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0.hdf
Worst quantile of [(3L, 2L)] at 4.73601800919
Data saved in ./results/iterative_save0_0_0_0_0_0_1_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0.hdf
Worst quantile of [(4L, 0L)] at 4.29181

Data saved in ./results/iterative_save1_0_1_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_1.hdf
Worst quantile of [(1L, 0L), (7L, 6L), (2L, 1L)] at 1.65398906471
Data saved in ./results/iterative_save1_0_0_1_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_1.hdf
Worst quantile of [(1L, 0L), (7L, 6L), (3L, 0L)] at 1.37856669144
Data saved in ./results/iterative_save1_0_0_0_1_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_1.hdf
Worst quantile of [(1L, 0L), (7L, 6L), (3L, 1L)] at 1.54632405095
Data saved in ./results/iterative_save1_0_0_0_0_1_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_1.hdf
Worst quantile of [(1L, 0L), (7L, 6L), (3L, 2L)] at 1.40091373338
Data saved in ./results/iterative_save1_0_0_0_0_0_1_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_1.hdf
Worst quantile of [(1L, 0L), (7L, 6L), (4L, 0L)] at 1.48910115172
Data saved in ./results/iterative_save1_0_0_0_0_0_0_1_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_1.hdf
Worst quantile of [(1L, 0L), (7L, 6L), (4L, 1L)] at 1.30136216062
Data saved in ./

In [25]:
families = np.zeros((dim, dim))
families[1, 0] = 1
families[6, 2] = 1
families[7, 6] = 1

quant_estimate = ConservativeEstimate(model_func=test_func, margins=margins, families=families)
res = quant_estimate.gridsearch_minimize(None, 20000, 'vertices', q_func=q_func)
res.min_quantity

1.4710180342208781